In [451]:
from sklearn.datasets import make_classification
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import ndjson
import json
import time
import pandas_gbq

In [426]:
#Create fake dataset

"""
The purpose in this section is to create a fake data set where we can simulate supply chain trailer data.
To get the data into into a workable form, we will need to create bins and specific labels for each column.

Features we will create:
    Sku
    inventory
    lead time
    forecast 3 months
    forecase 6 months
    sales 1 month
    sale 3 months
    sale 6 months
    potential issue - categorical
    pieces past due
    min_rec_stock
    backorder - output - is the item out of stock or no?
    
We will create an imbalance in our data by labeling only 5% of items out of stock
"""

X,y = make_classification(n_samples=1500000,
                          n_features=10,
                          n_informative=10,
                          n_redundant=0,
                          n_repeated=0,
                          n_classes=2, weights=[0.95,0.05],flip_y=0, random_state=32)

X = pd.DataFrame(X, columns=['inventory', 
                             'lead_time', 
                             'forecast_3_mon', 
                             'forecast_6_mon',
                             'sales_1_mon', 
                             'sales_3_mon', 
                             'sales_6_mon',
                             'potential_issue',
                             'pieces_past_due',
                             'min_rec_stock'
                            ])

#Relabel inventory - create 121 bins, label each, and confirm int type
inven_labels = [i for i in range(0,31)]
X['inventory'] = pd.cut(X['inventory'],31, labels=inven_labels)
X['inventory'] = X['inventory'].astype(int)

#Relabel lead times - create 10 bins, label each, confirm int type
lead_labels = [i for i in range(1,21)]
X['lead_time'] = pd.cut(X['lead_time'], 20, labels=lead_labels)
X['lead_time'] = X['lead_time'].astype(int)

#Relabel forecast 3 and 6 months - make distribution positive by adding random nums to the 3 month forecast
#Forecast 3 months
X['forecast_3_mon'] = [np.round(i+np.random.randint(10,50),0) for i in X['forecast_3_mon']]

#Forecast 6 months - multiply 3 month forecast by random num
X['forecast_6_mon'] = [i*np.random.randint(1,3) for i in X['forecast_3_mon']]

#Relabel previous sales for 1, 3, and 6 months
X['sales_1_mon'] = [np.round(i+np.random.randint(10,20),0) for i in X['sales_1_mon']]

#Sales 3 months
X['sales_3_mon'] = [i*np.random.randint(1,3) for i in X['sales_1_mon']]

#Sales 6 months
X['sales_6_mon'] = [i*np.random.randint(1,5) for i in X['sales_3_mon']]

#Potential issues
issues = (X['sales_1_mon'] > X['inventory']) & (X['lead_time'] < 30)
issues = issues.astype(str)

issue_list = []
for issue in issues:
    if issue == 'False':
        issue_list.append(0)
    else:
        issue_list.append(1)
        
X['potential_issue'] = issue_list

#Pieces past due
X['pieces_past_due'] = np.round([i*-1 if i < 0 else i for i in X['pieces_past_due']],0)

#Min rec stock - setting as a postive value based on sales data
min_stock=[]
for i,j in zip(X['min_rec_stock'], X['sales_1_mon']):
        if (i < 0) & (j > 10):
            min_stock.append((i*-1)*5)
        elif (i < 0) & (j < 10):
            min_stock.append((i*-1)+5)
        elif (i > 0) & (j > 10):
            min_stock.append(i*5)
        else:
            min_stock.append(i+5)

X['min_rec_stock'] = np.round(min_stock,0)

#Add in label for backorder
X['backorder'] = y

#Add sku id
X['sku'] = range(1,len(X['backorder']) + 1)

In [427]:
#Convert to dicts and save as json

"""
In this section, 5 tables will be created
    1. Inventory - will include sku, inventory, pieces past due, and min rec stock
    2. Sales - will include sku, sales 1 mon, sales 3 mon, and sales 6 mon
    3. Forecasts - will include sku, forecast 3 month, forecast 6 month
    4. Inventory ordering - will include sku, lead time, and potential issue
    5. Backorder - will include sku and if the item went on backorder
"""   
start = time.time()

#Inventory table
inventory = np.asanyarray(X[['sku', 'inventory', 'pieces_past_due', 'min_rec_stock']])
inventory_list=[]
for i in range(0,len(inventory)):
    subset = inventory[i]
    record = {'sku' : int(subset[0]),
             'inventory': int(subset[1]),
             'pieces_past_due': int(subset[2]),
             'min_rec_stock': int(subset[3])}
    inventory_list.append(record)
    
#Assert values match and length is the same
assert X['sku'][0] == inventory_list[0]['sku']
assert len(inventory_list) == len(X['sku']) 

#save as json (new-line delimited)
with open('inventory.ndjson', 'w') as f:
    ndjson.dump(inventory_list,f)
    
#Print that everything is saved
print('Inventory json saved')
    
#Sales table
sales = np.asanyarray(X[['sku','sales_1_mon', 'sales_3_mon', 'sales_6_mon']])
sales_list=[]
for i in range(0,len(sales)):
    subset = sales[i]
    record = {'sku' : int(subset[0]),
             'sales_1_mon': int(subset[1]),
             'sales_3_mon': int(subset[2]),
             'sales_6_mon': int(subset[3])}
    sales_list.append(record)
    
#Assert values match and length is the same
assert X['sku'][0] == sales_list[0]['sku']
assert len(sales_list) == len(X['sku'])

#save as json (new-line delimited)
with open('sales.ndjson', 'w') as f:
    ndjson.dump(sales_list,f)
    
#Print that everything is saved
print('Sales json saved')

#Forescast table
forecast = np.asanyarray(X[['sku','forecast_3_mon', 'forecast_6_mon']])
forecast_list=[]
for i in range(0,len(forecast)):
    subset = forecast[i]
    record = {'sku' : int(subset[0]),
             'forecast_3_mon': int(subset[1]),
             'forecast_6_mon': int(subset[2])}
    forecast_list.append(record)
    
#Assert values match and length is the same
assert X['sku'][0] == forecast_list[0]['sku']
assert len(forecast_list) == len(X['sku'])

#save as json (new-line delimited)
with open('forecast.ndjson', 'w') as f:
    ndjson.dump(forecast_list,f)
    
#Print that everything is saved
print('Forecast json saved')

#Inventory ordering
inventory_ordering = np.asanyarray(X[['sku','lead_time', 'potential_issue']])
inventory_ordering_list=[]   
for i in range(0,len(inventory_ordering)):
    subset = inventory_ordering[i]
    record = {'sku' : int(subset[0]),
             'lead_time': int(subset[1]),
             'potential_issue': int(subset[2])}
    inventory_ordering_list.append(record)
    
#Assert values match and length is the same
assert X['sku'][0] == inventory_ordering_list[0]['sku']
assert len(inventory_ordering_list) == len(X['sku'])

#save as json (new-line delimited)
with open('inventory_ordering.ndjson', 'w') as f:
    ndjson.dump(inventory_ordering_list,f)
    
#Print that everything is saved
print('Inventory ordering json saved')
    
#Backorder table    
backorder = np.asanyarray(X[['sku','backorder']])
backorder_list=[]   
for i in range(0,len(backorder)):
    subset = backorder[i]
    record = {'sku' : int(subset[0]),
             'backorder': int(subset[1])}
    backorder_list.append(record)
    
#Assert values match and length is the 
assert X['sku'][0] == backorder_list[0]['sku']
assert len(backorder_list) == len(X['sku'])  

#save as json (new-line delimited)
with open('backorder.ndjson', 'w') as f:
    ndjson.dump(backorder_list,f)
    
#Print that everything is saved
print('Backorder json saved')

end = time.time()
print("Total time was " + str(np.round((end-start)/60,2)) + " minutes")

Inventory json saved
Sales json saved
Forecast json saved
Inventory ordering json saved
Backorder json saved
Total time was 0.52 minutes


In [428]:
#Bigquery section

"""
1st - Call bigquery and create new dataset then tables
2nd - Upload jsons to bigquery
3rd - Query the data tables to pull in the necessary information
"""

#1 Call bigquery and create new dataset then tables

#Call BigQuery
from google.cloud import bigquery
from google.oauth2 import service_account

#credentials 
credentials = service_account.Credentials.from_service_account_file(
    '****_credentials.json')

#Project id name
project_id = '****'

#create the client
client = bigquery.Client(credentials= credentials,project=project_id)

#Create new dataset for project
DATASET_ID = 'backorder_chain_example'
dataset_ref = client.dataset(DATASET_ID)
dataset = bigquery.Dataset(dataset_ref)
dataset.description = 'my dataset'
dataset = client.create_dataset(dataset)

In [425]:
#Delete stuff from bigquery if needed
#client.delete_dataset('steven-data.backorder_chain_example', delete_contents=True)
#client.delete_table('steven-data.backorder_chain_example.inventory_ordering')
#client.delete_table('steven-data.backorder_chain_example.backorder')
#client.delete_table('steven-data.backorder_chain_example.forecast')
#client.delete_table('steven-data.backorder_chain_example.sales')

In [429]:
#Create the tables

#Create inventory table
dataset_ref = client.dataset('backorder_chain_example')  
table_ref = dataset_ref.table('inventory')
    
SCHEMA = [
    bigquery.SchemaField('sku', 'INTEGER', mode='NULLABLE'),
    bigquery.SchemaField('inventory', 'INTEGER', mode='NULLABLE'),
    bigquery.SchemaField('pieces_past_due', 'INTEGER', mode='NULLABLE'),
    bigquery.SchemaField('min_rec_stock', 'INTEGER', mode='NULLABLE'),
]
table = bigquery.Table(table_ref, schema=SCHEMA)
table = client.create_table(table)     

assert table.table_id == 'inventory'

#Create sales table
table_ref = dataset_ref.table('sales')
    
SCHEMA = [
    bigquery.SchemaField('sku', 'INTEGER', mode='NULLABLE'),
    bigquery.SchemaField('sales_1_mon', 'INTEGER', mode='NULLABLE'),
    bigquery.SchemaField('sales_3_mon', 'INTEGER', mode='NULLABLE'),
    bigquery.SchemaField('sales_6_mon', 'INTEGER', mode='NULLABLE'),
]
table = bigquery.Table(table_ref, schema=SCHEMA)
table = client.create_table(table)     

assert table.table_id == 'sales'

#Create forecast table
table_ref = dataset_ref.table('forecast')
    
SCHEMA = [
    bigquery.SchemaField('sku', 'INTEGER', mode='NULLABLE'),
    bigquery.SchemaField('forecast_3_mon', 'INTEGER', mode='NULLABLE'),
    bigquery.SchemaField('forecast_6_mon', 'INTEGER', mode='NULLABLE'),
]
table = bigquery.Table(table_ref, schema=SCHEMA)
table = client.create_table(table)     

assert table.table_id == 'forecast'

#Create inventory ordering
table_ref = dataset_ref.table('inventory_ordering')
    
SCHEMA = [
    bigquery.SchemaField('sku', 'INTEGER', mode='NULLABLE'),
    bigquery.SchemaField('lead_time', 'INTEGER', mode='NULLABLE'),
    bigquery.SchemaField('potential_issue', 'INTEGER', mode='NULLABLE'),
]
table = bigquery.Table(table_ref, schema=SCHEMA)
table = client.create_table(table)     

assert table.table_id == 'inventory_ordering'

#Create backorder table
table_ref = dataset_ref.table('backorder')
    
SCHEMA = [
    bigquery.SchemaField('sku', 'INTEGER', mode='NULLABLE'),
    bigquery.SchemaField('backorder', 'INTEGER', mode='NULLABLE'),
]
table = bigquery.Table(table_ref, schema=SCHEMA)
table = client.create_table(table)     

assert table.table_id == 'backorder'

In [430]:
#Upload jsons to Bigquery

start = time.time()

#Inventory upload
dataset = client.dataset('backorder_chain_example')
table = dataset.table('inventory')

with open('inventory.ndjson', 'rb') as source_file:
    job_config = bigquery.LoadJobConfig()
    job_config.source_format = 'NEWLINE_DELIMITED_JSON'
    job = client.load_table_from_file(
        source_file, table, job_config=job_config)
    
print('Inventory uploaded')
    
#Sales upload
table = dataset.table('sales')

with open('sales.ndjson', 'rb') as source_file:
    job_config = bigquery.LoadJobConfig()
    job_config.source_format = 'NEWLINE_DELIMITED_JSON'
    job = client.load_table_from_file(
        source_file, table, job_config=job_config)
    
print('Sales uploaded')
                      
#Forecast upload
table = dataset.table('forecast')

with open('forecast.ndjson', 'rb') as source_file:
    job_config = bigquery.LoadJobConfig()
    job_config.source_format = 'NEWLINE_DELIMITED_JSON'
    job = client.load_table_from_file(
        source_file, table, job_config=job_config)

print('Forecast uploaded')
    
#Inventory ordering upload
table = dataset.table('inventory_ordering')

with open('inventory_ordering.ndjson', 'rb') as source_file:
    job_config = bigquery.LoadJobConfig()
    job_config.source_format = 'NEWLINE_DELIMITED_JSON'
    job = client.load_table_from_file(
        source_file, table, job_config=job_config)
    
print('Inventory ordering uploaded')
    
#Backorder upload
table = dataset.table('backorder')

with open('backorder.ndjson', 'rb') as source_file:
    job_config = bigquery.LoadJobConfig()
    job_config.source_format = 'NEWLINE_DELIMITED_JSON'
    job = client.load_table_from_file(
        source_file, table, job_config=job_config)

print('Backorder uploaded')
    
end = time.time()
print("Total time was " + str(np.round((end-start)/60,2)) + " minutes")

Inventory uploaded
Sales uploaded
Forecast uploaded
Inventory ordering uploaded
Backorder uploaded
Total time was 1.03 minutes


In [443]:
#Query Bigquery to join tables and pull in data
sql = """
    SELECT A.sku, A.inventory, A.pieces_past_due, A.min_rec_stock,
      B.sales_1_mon, B.sales_3_mon, B.sales_6_mon,
      C.forecast_3_mon, C.forecast_6_mon,
      D.lead_time, D.potential_issue,
      E.backorder
FROM `backorder_chain_example.inventory` as A
INNER JOIN  `backorder_chain_example.sales`  as B
ON A.sku = B.sku
INNER JOIN `backorder_chain_example.forecast` as C
ON A.sku = C.sku
INNER JOIN `backorder_chain_example.inventory_ordering` as D
ON A.sku = D.sku
INNER JOIN `backorder_chain_example.backorder` as E
ON A.sku = E.sku
  """
#results = client.query(sql)
start = time.time()

df1 = pd.read_gbq(sql, dialect='standard', project_id=project_id)

end = time.time()
print("Query time was " + str(np.round((end-start)/60,2)) + " minutes")

Pandas-gbq total time was 2.11 minutes
GBQ total time was 2.28 minutes


In [ ]:
#Over sample data using smote
